In [30]:
import time
from typing import List, Optional, Any
import numpy as np
import matplotlib.pyplot as plt 

from docplex.mp.model import Model
from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import CobylaOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
from qiskit.optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer
from qiskit.optimization.converters import QuadraticProgramToIsing

In [2]:
cobyla = CobylaOptimizer()

qaoa = MinimumEigenOptimizer(QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator')))

exact = MinimumEigenOptimizer(NumPyMinimumEigensolver()) # to solve QUBOs

In [85]:
def carro_izquierda_up(x1,x2,x3):
    recomp = 5*(x3==0 & x2 ==0) - 3*(x2==1) - 3*(x3==1)
    return recomp

def carro_derecha_mitad(x1,x2,x3):
    recomp = -10*(x1==1)
    return recomp
    
def nada_abajo(x1,x2,x3):
    return 1

In [86]:
cuadricula = np.array([[1,0,0],
                       [1,0,0],
                       [0,0,0]])
print(carro_izquierda_up(*cuadricula[0,:]))
print(*cuadricula[0,:])

5
1 0 0


In [104]:
mdl = Model('derechaIzquierdaNada')

cuadricula = np.array([[0,0,1],
                       [1,0,0],
                       [0,0,0]])


u = mdl.binary_var(name='u')
d = mdl.binary_var(name='d')
l = mdl.binary_var(name='l')
r = mdl.binary_var(name='r')
stay = mdl.binary_var(name='stay')

mdl.minimize( -(carro_izquierda_up(*cuadricula[0,:])*u +carro_derecha_mitad(*cuadricula[1,:])*stay + nada_abajo(*cuadricula[2,:])*d ))
#mdl.add_constraint(u+d+l+r+stay == 1, "cons1")

# load quadratic program from docplex model
qp = QuadraticProgram()
qp.from_docplex(mdl)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: derechaIzquierdaNada

Minimize
 obj: 3 u - d + 10 stay
Subject To

Bounds
 0 <= u <= 1
 0 <= d <= 1
 0 <= l <= 1
 0 <= r <= 1
 0 <= stay <= 1

Binaries
 u d l r stay
End



In [89]:
# # define QUBO optimizer
# qubo_optimizer = exact 

# # define classical optimizer
# convex_optimizer = cobyla
# # convex_optimizer = cplex  # uncomment to use CPLEX instead

# # initialize ADMM with quantum QUBO optimizer and classical convex optimizer
# admm_q = ADMMOptimizer(params=admm_params, 
#                        qubo_optimizer=qubo_optimizer,
#                        continuous_optimizer=convex_optimizer)

In [90]:
# result_q = admm_q.solve(qp)

In [91]:
# print("x={}".format(result_q.x))
# print("fval={:.2f}".format(result_q.fval))

x=[0. 0. 0. 0. 0.]
fval=0.00


In [97]:
# plt.clf()
# plt.plot(result_q.state.residuals)
# plt.xlabel("Iterations")
# plt.ylabel("Residuals")
# plt.show()

In [98]:
# qp2op = QuadraticProgramToIsing()
# op, offset = qp2op.encode(qp)
# print('offset: {}'.format(offset))


In [105]:
from qiskit.optimization.converters import IsingToQuadraticProgram

In [106]:
op2qp = IsingToQuadraticProgram()
print(op2qp.encode(op, offset).export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: [ - 10 x_0^2 ]/2
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4
End



/opt/conda/lib/python3.7/site-packages/qiskit/optimization/converters/ising_to_quadratic_program.py:45: DeprecationWarning: The IsingToQuadraticProgram class is deprecated and will be removed in a future release. Use the .from_ising() method on the QuadraticProgram class instead.
  "instead.", DeprecationWarning)


In [107]:
qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator'))
exact_mes = NumPyMinimumEigensolver()

In [108]:
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [109]:
exact_result = exact.solve(qp)
print(exact_result)

optimal function value: -1.0
optimal value: [0. 1. 1. 1. 0.]
status: SUCCESS


In [110]:
qaoa_result = qaoa.solve(qp)
print(qaoa_result)

optimal function value: -1.0
optimal value: [0. 1. 0. 0. 0.]
status: SUCCESS


In [113]:
op

PauliOp(Pauli(z=[True, False, False, False, False], x=[False, False, False, False, False]), coeff=2.5)